<a href="https://colab.research.google.com/github/Ayman-Abdeen/chatboot-/blob/main/CNN/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer

import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary


In [2]:
nltk.__version__ 

'3.2.5'

In [3]:
torch.__version__

'1.10.0+cu111'

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
stemmer = PorterStemmer()

tokenize --> lower+stem --> exclude punctuation characters --> bag of words


Tokenize

In [6]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)


In [7]:
def stem(word):
    return stemmer.stem(word.lower())


In [8]:
def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [9]:
a="split sentence into array of words,tokens"
print(a)
a=tokenize(a)
print(a)

split sentence into array of words,tokens
['split', 'sentence', 'into', 'array', 'of', 'words', ',', 'tokens']


In [10]:
words = ["Organize", "organizes", "organizing"]
stemmed_words = [stem(w) for w in words]
print(stemmed_words)

['organ', 'organ', 'organ']


Create training data

In [11]:
with open('intents.json', 'r') as f:
    intents = json.load(f)


In [12]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'funny', 'patterns': ['Tell me a joke!', 'Tell me something funny!', 'Do you know a joke?'], 'responses': ['Why did the hipster burn his mouth? He drank the coffee before it was cool.', 'What did the buffalo say when his son left for college? Bison.']}, {'tag': 'Cuts', 'patterns': ['What to do if Cuts?', 'How to cure Cuts?', 'Which medicine to apply for Cuts?', 'what to apply on cuts?', 'Cuts'], 'responses': ['Wash the cut pro

In [13]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

In [ ]:
all_words

In [ ]:
xy

In [16]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

204 patterns
48 tags: ['Abdonominal Pain', 'Abrasions', 'Broken Toe', 'Bruises', 'CPR', 'Chemical Burn', 'Choking', 'Cold', 'Cough', 'Cuts', 'Diarrhea', 'Drowning', 'Eye Injury', 'Fainting', 'Fever', 'Fracture', 'Frost bite', 'Gastrointestinal problems', 'Head Injury', 'Headache', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites', 'Nasal Congestion', 'Normal Bleeding', 'Poison', 'Pulled Muscle', 'Rash', 'Rectal bleeding', 'Skin problems', 'Sore Throat', 'Splinter', 'Sprains', 'Strains', 'Sun Burn', 'Teeth', 'Testicle Pain', 'Vertigo', 'Wound', 'animal bite', 'funny', 'goodbye', 'greeting', 'nose bleed', 'seizure', 'snake bite', 'stings', 'thanks']
135 unique stemmed words: ["'s", 'a', 'abdonomin', 'abras', 'allergi', 'am', 'an', 'anim', 'anyon', 'appli', 'are', 'better', 'bit', 'bite', 'bitten', 'bleed', 'block', 'bring', 'broken', 'bruis', 'burn', 'by', 'bye', 'caus', 'chemic', 'choke', 'cold', 'congest', 'cough', 'cpr', 'cream', 'cure', 'cut', 'day', 'diagnos', 'diarrhea', 'do', 'doe'

In [17]:

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [18]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [19]:
y_train

array([42, 42, 42, 42, 42, 42, 41, 41, 41, 47, 47, 47, 47, 40, 40, 40,  9,
        9,  9,  9,  9,  1,  1,  1,  1,  1,  1, 46, 46, 46, 46, 31, 31, 31,
       31, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 14, 14, 14, 14, 23, 23,
       23, 23, 23, 23,  8,  8,  8,  8,  8, 30, 30, 30, 30, 17, 17, 17, 17,
       29, 29, 29, 29,  0,  0,  0,  0,  3,  3,  3,  3,  2,  2,  2,  2,  6,
        6,  6,  6, 38, 38, 38, 38, 10, 10, 10, 10, 16, 16, 16, 16, 20, 20,
       20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 43, 43, 43, 43, 26, 26, 26,
       26, 28, 28, 28, 28, 34, 34, 34, 34, 36, 36, 36, 36, 37, 37, 37, 37,
       24, 24, 24, 24, 12, 12, 12, 12,  5,  5,  5,  5, 25, 25, 25, 25, 35,
       35, 35, 35, 44, 44, 44, 44, 18, 18, 18, 18, 13, 13, 13, 13, 19, 19,
       19, 19,  7,  7,  7,  7, 27, 27, 27, 27, 45, 45, 45, 45, 45, 39, 39,
       39, 39, 39, 39, 39, 11, 11, 11, 11,  4,  4,  4,  4, 15, 15, 15, 15])

In [20]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)


135 48


In [21]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


In [22]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,num_workers=0)

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [25]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [26]:
print(model)

NeuralNet(
  (l1): Linear(in_features=135, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=48, bias=True)
  (relu): ReLU()
)


In [36]:
summary(model,(1, input_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 8]           1,088
              ReLU-2                 [-1, 1, 8]               0
            Linear-3                 [-1, 1, 8]              72
              ReLU-4                 [-1, 1, 8]               0
            Linear-5                [-1, 1, 48]             432
Total params: 1,592
Trainable params: 1,592
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [28]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')



Epoch [100/1000], Loss: 0.4457
Epoch [200/1000], Loss: 0.0198
Epoch [300/1000], Loss: 0.0010
Epoch [400/1000], Loss: 0.0002
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000


In [30]:
#save_model
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


===============================================================================

In [31]:
FILE = "data.pth"
data = torch.load(FILE)

In [32]:

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]


In [33]:

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


NeuralNet(
  (l1): Linear(in_features=135, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=48, bias=True)
  (relu): ReLU()
)

In [34]:
bot_name = "Aymon"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.60:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hi
Aymon: Hi there, what can I do for you?
You: How do I handle Cuts
Aymon: Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.
You: my muscle is pulled 
Aymon: 1) Apply ice for 10 to 15 minutes every 1 hour for the first day and every 3 to 4 hours after that. Use ice for the first 3 days. After 3 days, either heat or ice may be helpful if you still have pain. Rest the pulled muscle for at least a day. 
You: Had an accident that caused Abrasions
Aymon: Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use 